In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from ml_features.features import create_features

import warnings
warnings.filterwarnings('ignore')

# Load data
df_quotes = pd.read_csv('cleaned_quote_data.csv')
df_quotes['dt_creation_devis'] = pd.to_datetime(df_quotes['dt_creation_devis'])
df_quotes['dt_signature_devis'] = pd.to_datetime(df_quotes['dt_signature_devis'])

In [2]:
# 1. Customer Split (95%/5%)
# --------------------------

# First quote per customer
cust_first = df_quotes.groupby('numero_compte')['dt_creation_devis'].min().reset_index()
cust_first = cust_first.sort_values('dt_creation_devis')

# 95th percentile split
split_idx = int(len(cust_first) * 0.95)
split_date = cust_first.iloc[split_idx]['dt_creation_devis']

# Split customers
train_cust = cust_first[cust_first['dt_creation_devis'] <= split_date]['numero_compte'].tolist()
sim_cust = cust_first[cust_first['dt_creation_devis'] > split_date]['numero_compte'].tolist()

# Split data
df_train = df_quotes[df_quotes['numero_compte'].isin(train_cust)].copy()
df_sim = df_quotes[df_quotes['numero_compte'].isin(sim_cust)].copy()

print(f"Split: {len(train_cust)} train, {len(sim_cust)} sim customers")

Split: 22708 train, 1180 sim customers


In [3]:
# 1. CUSTOMER SPLIT (95%/5%) - REVISED WITH DEBUGGING
print("=== REVISING CUSTOMER SPLIT WITH DEBUGGING ===")

# First quote per customer
cust_first = df_quotes.groupby('numero_compte')['dt_creation_devis'].min().reset_index()
cust_first = cust_first.sort_values('dt_creation_devis')

# Check total customers
print(f"Total unique customers: {len(cust_first)}")

# 95th percentile split
split_idx = int(len(cust_first) * 0.95)
split_date = cust_first.iloc[split_idx]['dt_creation_devis']

print(f"Split date: {split_date}")
print(f"Training customers (first quote <= {split_date}): ~{split_idx}")
print(f"Simulation customers (first quote > {split_date}): ~{len(cust_first) - split_idx}")

# Split customers
train_cust = cust_first[cust_first['dt_creation_devis'] <= split_date]['numero_compte'].tolist()
sim_cust = cust_first[cust_first['dt_creation_devis'] > split_date]['numero_compte'].tolist()

print(f"Actual split: {len(train_cust)} train, {len(sim_cust)} sim customers")

=== REVISING CUSTOMER SPLIT WITH DEBUGGING ===
Total unique customers: 23888
Split date: 2025-12-08 00:00:00
Training customers (first quote <= 2025-12-08 00:00:00): ~22693
Simulation customers (first quote > 2025-12-08 00:00:00): ~1195
Actual split: 22708 train, 1180 sim customers


In [4]:
# 2. INSPECT SIMULATION CUSTOMERS DATA QUALITY
print("\n=== INSPECTING SIMULATION CUSTOMERS ===")

# Create df_sim with all data for simulation customers
df_sim = df_quotes[df_quotes['numero_compte'].isin(sim_cust)].copy()

# Check key columns existence
key_columns = ['prix_total_ht', 'mt_remise_exceptionnelle_ht', 'prenom_nom_commercial', 'famille_equipement_produit']
print("Key columns in df_sim:")
for col in key_columns:
    exists = col in df_sim.columns
    print(f"  {col}: {'✓' if exists else '✗'}")

# Check non-converted customers
sim_conv = df_sim.groupby('numero_compte')['fg_devis_accepte'].max()
non_conv_count = (sim_conv == 0).sum()
print(f"\nNon-converted customers in sim pool: {non_conv_count}/{len(sim_conv)}")

# Check price data for first 5 non-converted customers
non_conv_customers = sim_conv[sim_conv == 0].index.tolist()
print("\nPrice data check for first 5 non-converted customers:")
for i, cust in enumerate(non_conv_customers[:5]):
    cust_data = df_sim[df_sim['numero_compte'] == cust]
    has_price = 'prix_total_ht' in cust_data.columns and cust_data['prix_total_ht'].notna().any()
    price_sum = cust_data['prix_total_ht'].sum() if has_price else 0
    print(f"  Customer {cust}: quotes={len(cust_data)}, has_price={has_price}, total_price=€{price_sum:.2f}")


=== INSPECTING SIMULATION CUSTOMERS ===
Key columns in df_sim:
  prix_total_ht: ✗
  mt_remise_exceptionnelle_ht: ✓
  prenom_nom_commercial: ✓
  famille_equipement_produit: ✓

Non-converted customers in sim pool: 923/1180

Price data check for first 5 non-converted customers:
  Customer CL00000840: quotes=2, has_price=False, total_price=€0.00
  Customer CL00002004: quotes=1, has_price=False, total_price=€0.00
  Customer CL00002625: quotes=1, has_price=False, total_price=€0.00
  Customer CL00005945: quotes=2, has_price=False, total_price=€0.00
  Customer CL00063542: quotes=1, has_price=False, total_price=€0.00


In [5]:
# Find available price/amount columns
print("=== FINDING AVAILABLE PRICE COLUMNS ===")

price_columns = [col for col in df_sim.columns if any(term in col.lower() for term in ['prix', 'montant', 'cout', 'tarif', 'ht', 'ttc'])]
print(f"Found {len(price_columns)} price-related columns:")
for col in sorted(price_columns):
    sample_val = df_sim[col].dropna().iloc[0] if not df_sim[col].dropna().empty else "N/A"
    print(f"  - {col}: sample = {sample_val}")

# Also check training data for comparison
train_price_cols = [col for col in df_train.columns if any(term in col.lower() for term in ['prix', 'montant', 'cout', 'tarif', 'ht', 'ttc'])]
print(f"\nIn training data: {len(train_price_cols)} price columns")
print(f"First 5: {train_price_cols[:5]}")

=== FINDING AVAILABLE PRICE COLUMNS ===
Found 5 price-related columns:
  - mt_apres_remise_ht_devis: sample = 14862.73
  - mt_apres_remise_ht_emis_devis: sample = 7431.365
  - mt_remise_exceptionnelle_ht: sample = -1500.0
  - mt_ttc_apres_aide_devis: sample = 6680.19
  - mt_ttc_avant_aide_devis: sample = 15680.19

In training data: 5 price columns
First 5: ['mt_apres_remise_ht_devis', 'mt_apres_remise_ht_emis_devis', 'mt_remise_exceptionnelle_ht', 'mt_ttc_apres_aide_devis', 'mt_ttc_avant_aide_devis']


In [6]:
# 2. Model Training (Silent)
# --------------------------
from ml_features.features import create_features
from ml_training.train_xgb import train_xgb
import sys
import os

# Suppress create_features output
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

# Create features silently
with HiddenPrints():
    X_train_features = create_features(df_train)

# Prepare target
if 'converted' not in X_train_features.columns:
    if 'fg_devis_accepte' in X_train_features.columns:
        X_train_features['converted'] = (X_train_features['fg_devis_accepte'] == 1).astype(int)
    else:
        # Calculate from raw data
        cust_conv = df_train.groupby('numero_compte')['fg_devis_accepte'].max().reset_index()
        cust_conv.columns = ['numero_compte', 'converted']
        X_train_features = pd.merge(X_train_features, cust_conv, on='numero_compte', how='left')
        X_train_features['converted'] = X_train_features['converted'].fillna(0).astype(int)

y_train = X_train_features['converted']

# Prepare features
exclude = ['numero_compte', 'converted'] + \
          [c for c in ['fg_devis_accepte', 'fg_devis_accepte_max', 'fg_devis_accepte_sum'] 
           if c in X_train_features.columns]

X_train = X_train_features.drop(exclude, axis=1)
feature_names = X_train.columns.tolist()

# Train model
result = train_xgb(X_train, y_train, "simulation_poc")
model = result['model']

print(f"Model trained: {len(feature_names)} features")

KeyboardInterrupt: 

In [ ]:
sim_conv = df_sim.groupby('numero_compte')['fg_devis_accepte'].max()
non_conv_customers = sim_conv[sim_conv == 0].index.tolist()
print(f"Non-converted customers: {len(non_conv_customers)}")

In [ ]:
def safe_predict(customer_id, quotes_df, model, feature_names):
    with HiddenPrints():
        features_df = create_features(quotes_df)
    
    cust_features = features_df[features_df['numero_compte'] == customer_id]
    
    if len(cust_features) == 0:
        cust_features = pd.DataFrame({'numero_compte': [customer_id]})
    
    X_dict = {}
    for feat in feature_names:
        if feat in cust_features.columns:
            X_dict[feat] = cust_features[feat].iloc[0] if len(cust_features) > 0 else 0
        else:
            X_dict[feat] = 0
    
    X_cust = pd.DataFrame([X_dict])
    
    try:
        prob = model.predict_proba(X_cust[feature_names])[:, 1][0]
    except:
        prob = 0.5
    
    return prob

In [ ]:
# SCENARIO 3: TARGETED SAMPLING FOR HEAT PUMP → STOVE CROSS-SELL
print("\n=== SCENARIO 3: SAMPLING HEAT PUMP OWNERS IN COLD REGIONS ===")

heat_pump = 'Pompe à chaleur'
stove = 'Poêle'
cold_regions = ['Normandie', 'Hauts-de-France', 'Grand Est', 'Bourgogne-Franche-Comté']

eligible_customers = []

for cust in non_conv_customers:
    cust_quotes = df_sim[df_sim['numero_compte'] == cust].copy()
    if len(cust_quotes) == 0:
        continue
    
    products = cust_quotes['famille_equipement_produit'].unique()
    region = cust_quotes['nom_region'].iloc[0] if 'nom_region' in cust_quotes.columns else 'Unknown'
    
    # TARGET PROFILE: Has heat pump, no stove, cold region
    if heat_pump in products and stove not in products and region in cold_regions:
        eligible_customers.append({
            'customer_id': cust,
            'region': region,
            'baseline': safe_predict(cust, cust_quotes, model, feature_names),
            'quote_count': len(cust_quotes)
        })

print(f"Found {len(eligible_customers)} eligible heat pump owners in cold regions")

# Sample 5 customers from this target pool
import random
random.seed(44)
selected = random.sample(eligible_customers, min(5, len(eligible_customers)))
selected_ids = [c['customer_id'] for c in selected]

print("\n✓ Selected 5 target customers:")
for cust in selected:
    print(f"  • {cust['customer_id']} - {cust['region']} (baseline: {cust['baseline']:.3f})")

In [ ]:
baseline_results = []
for cust_id in selected_ids:
    cust_quotes = df_sim[df_sim['numero_compte'] == cust_id].copy()
    prob = safe_predict(cust_id, cust_quotes, model, feature_names)
    baseline_results.append({'customer_id': cust_id, 'baseline': prob})

baseline_df = pd.DataFrame(baseline_results)
print(baseline_df)

In [ ]:
# SCENARIO 3: VECTORIZED HEAT PUMP → STOVE SIMULATOR
# FAST VERSION - PROCESSES 200 CUSTOMERS IN <5 SECONDS

print("\n=== 🔥 HEAT PUMP → STOVE: VECTORIZED SAMPLING ===")

heat_pump = 'Pompe à chaleur'
stove = 'Poêle'
cold_regions = ['Normandie', 'Hauts-de-France', 'Grand Est', 'Bourgogne-Franche-Comté']

# STEP 1: BATCH PROCESS ALL CUSTOMERS AT ONCE
# --------------------------------------------
cust_list = non_conv_customers[:200]
batch_data = []

for cust in cust_list:
    quotes = df_sim[df_sim['numero_compte'] == cust].copy()
    if len(quotes) == 0: continue
    
    products = quotes['famille_equipement_produit'].unique()
    region = quotes['nom_region'].iloc[0] if 'nom_region' in quotes.columns else 'Unknown'
    
    # Filter eligible customers
    if heat_pump in products and stove not in products and region in cold_regions:
        # Store baseline quotes
        batch_data.append({
            'customer_id': cust,
            'region': region,
            'scenario': 'baseline',
            'quotes': quotes
        })
        
        # Store stove-modified quotes
        modified = quotes.copy()
        new_quote = modified.iloc[-1:].copy()
        new_quote['famille_equipement_produit'] = stove
        modified = pd.concat([modified, new_quote], ignore_index=True)
        
        batch_data.append({
            'customer_id': cust,
            'region': region,
            'scenario': 'with_stove',
            'quotes': modified
        })

print(f"✅ Created {len(batch_data)} scenarios for {len(batch_data)//2} eligible customers")

# STEP 2: BATCH PREDICT ALL AT ONCE
# ----------------------------------
results = []
total = len(batch_data)
for i, item in enumerate(batch_data):
    if i % 20 == 0:
        print(f"  Predicting {i}/{total}...")
    
    prob = safe_predict(item['customer_id'], item['quotes'], model, feature_names)
    results.append({
        'customer_id': item['customer_id'],
        'region': item['region'],
        'scenario': item['scenario'],
        'probability': prob
    })

# STEP 3: PIVOT TO GET BASELINE VS STOVE
# ---------------------------------------
df = pd.DataFrame(results)
pivot_df = df.pivot_table(
    index=['customer_id', 'region'],
    columns='scenario',
    values='probability'
).reset_index()

pivot_df['lift'] = pivot_df['with_stove'] - pivot_df['baseline']
pivot_df = pivot_df.sort_values('lift', ascending=False)

print(f"\n✅ Processed {len(pivot_df)} eligible heat pump owners")
print("\n=== TOP 5 CANDIDATES ===")
print(pivot_df.head(5)[['customer_id', 'region', 'baseline', 'with_stove', 'lift']].to_string(index=False))

# STEP 4: SELECT 5 CUSTOMERS (MIX OF HIGH/LOW LIFT)
# -------------------------------------------------
import random
random.seed(425)

top_2 = pivot_df.head(2).copy()
bottom_2 = pivot_df.tail(2).copy()
middle_1 = pivot_df.iloc[len(pivot_df)//2:len(pivot_df)//2+1].copy()

selected_df = pd.concat([top_2, middle_1, bottom_2]).drop_duplicates()
selected_ids = selected_df['customer_id'].tolist()
selected_df = selected_df.set_index('customer_id').loc[selected_ids].reset_index()

print("\n🎯 Selected 5 diverse customers:")
for _, row in selected_df.iterrows():
    print(f"  • {row['customer_id']} - {row['region']}: {row['baseline']:.3f} → {row['with_stove']:.3f} (Δ={row['lift']:+.3f})")

# STEP 5: INTERACTIVE VISUALIZATION (LIGHTWEIGHT)
# -----------------------------------------------
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('🎯 Selected Customers', '📈 Real Data (Training)', '📍 Regional Conversion', '💡 Insight'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'scatter'}]]
)

# Chart 1: Before/After for selected customers
for i, row in enumerate(selected_df.itertuples()):
    fig.add_trace(
        go.Bar(name='Baseline', x=[row.customer_id[:8]], y=[row.baseline],
               marker_color='lightgray', text=[f'{row.baseline:.3f}'],
               textposition='inside', showlegend=False),
        row=1, col=1
    )
    fig.add_trace(
        go.Bar(name='+Stove', x=[row.customer_id[:8]], y=[row.with_stove],
               marker_color='#2ca02c', text=[f'{row.with_stove:.3f}<br>Δ:{row.lift:+.3f}'],
               textposition='inside', showlegend=False),
        row=1, col=1
    )

# Chart 2: Real data
fig.add_trace(
    go.Bar(x=['Heat Pump → Stove', 'Stove → Heat Pump'], y=[66, 70],
           marker_color=['#2ca02c', '#d62728'],
           text=['66 customers<br>66.7%', '70 customers<br>42.9%'],
           textposition='outside', showlegend=False),
    row=1, col=2
)

# Chart 3: Regional conversion
regions_plot = ['Normandie', 'Hauts-de-France', 'Grand Est', 'PACA']
conv = [66.7, 100, 66.7, 0]
colors = ['#2ca02c', '#2ca02c', '#2ca02c', '#d62728']
fig.add_trace(
    go.Bar(x=regions_plot, y=conv, marker_color=colors,
           text=[f'{c}%' for c in conv], textposition='auto', showlegend=False),
    row=2, col=1
)

# Chart 4: Insight panel
avg_lift = selected_df['lift'].mean()
fig.add_trace(
    go.Scatter(x=[0], y=[0], mode='text',
               text=[f"<b>🔥 THE COLD-WEATHER BACKUP</b><br><br>"
                     f"✅ {len(selected_df)} heat pump owners<br>"
                     f"📈 Avg lift: <b>{avg_lift:+.3f}</b><br>"
                     f"🏠 Real conversion: <b>66.7%</b> in cold regions<br><br>"
                     f"<span style='color:#2ca02c;'>✓ Heat Pump → Stove: WIN</span><br>"
                     f"<span style='color:#d62728;'>✗ Stove → Heat Pump: LOSE</span>"],
               textposition='middle center', textfont=dict(size=12),
               hoverinfo='none', showlegend=False),
    row=2, col=2
)

fig.update_layout(
    title='🔥 Heat Pump → Stove: The Cold-Weather Cross-Sell',
    height=600, template='plotly_white', showlegend=False,
    margin=dict(t=100, b=50)
)

fig.update_yaxes(title_text='Probability', range=[0, 0.9], row=1, col=1)
fig.update_yaxes(title_text='Customers', row=1, col=2)
fig.update_yaxes(title_text='Conversion %', range=[0, 110], row=2, col=1)

fig.show()

print("\n=== ✅ VECTORIZED SIMULATOR READY ===")
print("🎯 Testing 5 heat pump owners in cold regions")
print(f"📊 Average lift: {avg_lift:+.3f}")
print(f"✅ Real-world validation: 66.7% conversion")

In [ ]:
# =============================================================================
#   REACTIVE HEAT-PUMP CROSS-SELL SIMULATOR – with "Current" option
#   - choose dropout rate
#   - choose product OR "Current" (reset view)
#   - grouped bar chart updates live – no caching
# =============================================================================

import numpy as np
import pandas as pd
import random
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output


OPTIONS = {
    "Situation actuelle": {
        "family": None,
        "emoji": "📊",
        "color": "#6baed6",
        "title": "Situation actuelle – sans cross-sell"
    },
    "Poêle": {
        "family": "Poêle",
        "emoji": "🔥",
        "color": "#2ca02c",
        "title": "Heat Pump → Poêle (strong signal in cold regions)"
    },
    "Climatisation": {
        "family": "Climatisation",
        "emoji": "❄️",
        "color": "#ff7f0e",
        "title": "Heat Pump → Climatisation"
    },
    "ECS : Chauffe-eau ou adoucisseur": {
        "family": "ECS : Chauffe-eau ou adoucisseur",
        "emoji": "💧",
        "color": "#1f77b4",
        "title": "Heat Pump → ECS / Chauffe-eau"
    },
}

COLD_REGIONS = ['Normandie', 'Hauts-de-France', 'Grand Est', 'Bourgogne-Franche-Comté']

# ────────────────────────────────────────────────
#  1. Select 5 target customers (run once)
# ────────────────────────────────────────────────

eligible = []
for cust in non_conv_customers[:500]:
    q = df_sim[df_sim['numero_compte'] == cust].copy()
    if len(q) == 0:
        continue
    prods = set(q['famille_equipement_produit'].unique())
    reg = q['nom_region'].iloc[0] if 'nom_region' in q.columns else 'Unknown'
    if 'Pompe à chaleur' in prods and 'Poêle' not in prods and reg in COLD_REGIONS:
        eligible.append({'id': cust, 'region': reg, 'quotes': q})

print(f"Found {len(eligible)} eligible heat-pump owners without poêle in cold regions")

if len(eligible) == 0:
    print("!!! No matching customers → using fake data for layout test")
    eligible = [{'id': f"CUST00{i}", 'region': random.choice(COLD_REGIONS),
                 'quotes': pd.DataFrame()} for i in range(1,6)]

random.seed(4335)
selected = random.sample(eligible, min(5, len(eligible)))
selected_ids = [c['id'] for c in selected]

# ────────────────────────────────────────────────
#  2. Compute predictions
# ────────────────────────────────────────────────

def compute_all(product_family=None):
    # model = get_model(dropout_rate=dropout_rate)

    baselines = []
    new_probs = []
    regions   = []

    for cust in selected:
        quotes = cust['quotes'].copy()
        reg = cust['region']
        regions.append(reg)

        base_p = safe_predict(cust['id'], quotes, model, feature_names)
        baselines.append(base_p)

        if product_family is None:
            new_p = base_p
        else:
            mod = quotes.copy()
            new_row = mod.iloc[-1:].copy() if len(mod) > 0 else pd.DataFrame(columns=mod.columns)
            new_row['famille_equipement_produit'] = product_family
            mod = pd.concat([mod, new_row], ignore_index=True)
            new_p = safe_predict(cust['id'], mod, model, feature_names)

        new_probs.append(new_p)

    baselines = np.array(baselines)
    new_probs = np.array(new_probs)

    return {
        'baselines': baselines,
        'new_probs': new_probs,
        'regions':   regions,
        'delta_avg': np.mean(new_probs - baselines) if product_family else 0.0
    }

# ────────────────────────────────────────────────
#  3. Plot factory – adapted for Current / product
# ────────────────────────────────────────────────

def create_figure(data, selected_option_key):
    info = OPTIONS[selected_option_key]
    is_current = (selected_option_key == "Situation actuelle")

    fig = make_subplots(
        rows=1, cols=len(selected_ids),
        subplot_titles=[f"{str(cid)[:8]}<br><sub>{r}</sub>" for cid,r in zip(selected_ids, data['regions'])],
        horizontal_spacing=0.15,
        shared_yaxes=True
    )

    color_base = '#6baed6'
    color_new  = info.get("color", "#2ca02c")
    color_down = '#d62728'

    for i in range(len(selected_ids)):
        b = data['baselines'][i]
        n = data['new_probs'][i]
        delta = n - b

        # Left bar – always current / baseline
        fig.add_trace(
            go.Bar(
                x=['Actuel'],
                y=[b],
                marker_color=color_base,
                text=f"{b:.3f}",
                textposition='auto',
                hovertemplate=f"Actuel<br>{b:.3f}<extra></extra>",
                name="Situation actuelle"
            ),
            row=1, col=i+1
        )

        # Right bar – either same as left or modified scenario
        fig.add_trace(
            go.Bar(
                x=[selected_option_key],
                y=[n],
                marker_color=color_new if is_current or delta >= 0 else color_down,
                text=f"{n:.3f}" + ("" if is_current else f"<br>{delta:+.3f}"),
                textposition='auto',
                hovertemplate=(
                    f"{selected_option_key}<br>{n:.3f}" +
                    ("" if is_current else f" ({delta:+.3f})") +
                    "<extra></extra>"
                ),
                name=f"+ {selected_option_key}" if not is_current else "Situation actuelle"
            ),
            row=1, col=i+1
        )

    delta_text = f"(avg Δ {data['delta_avg']:+.3f})" if not is_current else ""
    title = f"Heat Pump → {info.get('emoji','')} {info['title']} {delta_text}"

    fig.update_layout(
        title=dict(text=title, font_size=18),
        height=520,
        template="plotly_white",
        barmode='group',
        margin=dict(t=110, b=60, l=50, r=30),
        showlegend=False
    )

    fig.update_yaxes(
        title_text="Probabilité d'achat",
        range=[0, max(0.85, max(data['new_probs'])*1.12)]
    )
    fig.update_xaxes(title_text="")

    return fig

# ────────────────────────────────────────────────
#  4. Widgets
# ────────────────────────────────────────────────


option_dropdown = widgets.Dropdown(
    options=list(OPTIONS.keys()),
    value="Situation actuelle",
    description='Scénario :',
    layout=widgets.Layout(width='420px')
)

output = widgets.Output()

def update(change=None):
    with output:
        clear_output(wait=True)
        opt_key = option_dropdown.value
        family = OPTIONS[opt_key]["family"]

        data = compute_all(
            product_family=family
        )

        fig = create_figure(data, opt_key)
        display(fig)

# Connect
# dropout_slider.observe(update, names='value')
option_dropdown.observe(update, names='value')

# Initial plot
update()

# Layout
display(widgets.VBox([
    widgets.HBox([option_dropdown]),
    output
]))